In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import structural_similarity as ssim

from util import get_parking_spots_bboxes, empty_or_not

/home/ahmedcr7/.local/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.1.3 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
def get_diff(img1, img2):
    return np.abs(np.mean(img1)-np.mean(img2))

In [5]:
cap = cv2.VideoCapture('parking_1920_1080.mp4')

mask = cv2.imread('mask_1920_1080.png',0)
connected_components = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)

bbox = get_parking_spots_bboxes(connected_components)
counter = 0
predictions = [None for i in bbox]
diff = [None for i in bbox]

prev_frame = None
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if counter % 50 == 0:
        if prev_frame is None: # NumPy arrays can't be directly compared to None using the equality operator (==)
            for (idx,(x, y, w, h)) in enumerate(bbox):
                roi = frame[y:y + h, x:x + w]
                pred = empty_or_not(roi)
                predictions[idx] = pred
        
        else:
            for (idx,(x, y, w, h)) in enumerate(bbox):
                roi = frame[y:y + h, x:x + w]
                prev_roi = prev_frame[y:y + h, x:x + w]
                diff[idx] = get_diff(roi, prev_roi)
            
            indices_of_changed_roi = [j for j in np.argsort(diff) if diff[j]/np.amax(diff) > 0.4]

            for idx in indices_of_changed_roi:
                x,y,w,h = bbox[idx]
                roi = frame[y:y + h, x:x + w]
                pred = empty_or_not(roi)
                predictions[idx] = pred

    for (idx,(x, y, w, h)) in enumerate(bbox):
        if predictions[idx]:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

    counter += 1
    prev_frame = frame

cap.release()

# cv2.imshow('mask', mask)
# cv2.waitKey(0)
cv2.destroyAllWindows()